In [1]:
import pandas as pd
from pathlib import Path
import pandas_datareader.data as reader
import datetime as dt
from MCForecastTools import MCSimulation

# For visualizations:
import seaborn as sns

# PART I: DATA CAPTURE AND ORGANIZATION

In [2]:
end = dt.date(2021, 12, 31)
start = dt.date(2020, 1, 1)

ticker1 = ["BTC-USD"]
ticker2 = ["IWP"]  #iShares Russell Mid-Cap GROWTH ETF, a midcap growth fund. Chose midcap to factor out size.
ticker3 = ["IWS"]  #iShares Russell Mid-Cap VALUE ETF, a midcap value fund. Chose midcap to factor out size.
ticker4 = ["MGC"]   #Vanguard Mega Cap ETF, a fund with LARGE CAPS only to feature size (large cap end of the spectrum).
ticker5 = ["IWC"]  #iShares Micro-Cap ETF, a fund with SMALL CAPS only to feature size (small cap end of the spectrum).

In [3]:
# Get BTC closing prices

btc_prices = reader.get_data_yahoo(ticker1, start, end)
display(btc_prices.head())
btc_prices.tail()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2020-01-01,7200.174316,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
2020-01-02,6985.470215,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2020-01-03,7344.884277,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
2020-01-04,7410.656738,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
2020-01-05,7411.317383,7411.317383,7544.497070,7400.535645,7410.451660,19725074095


Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2021-12-28,47588.855469,47588.855469,50679.859375,47414.210938,50679.859375,33430376883
2021-12-29,46444.710938,46444.710938,48119.742188,46201.496094,47623.871094,30049226299
2021-12-30,47178.125000,47178.125000,47879.964844,46060.312500,46490.605469,26686491018
2021-12-31,46306.445312,46306.445312,48472.527344,45819.953125,47169.371094,36974172400
2022-01-01,47686.812500,47686.812500,47827.312500,46288.484375,46311.746094,24582667004


In [4]:
btc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
btc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-01,7200.174316,7200.174316,7254.330566,7174.944336,7194.892090,18565664997
2020-01-02,6985.470215,6985.470215,7212.155273,6935.270020,7202.551270,20802083465
2020-01-03,7344.884277,7344.884277,7413.715332,6914.996094,6984.428711,28111481032
2020-01-04,7410.656738,7410.656738,7427.385742,7309.514160,7345.375488,18444271275
2020-01-05,7411.317383,7411.317383,7544.497070,7400.535645,7410.451660,19725074095


In [5]:
btc_prices = btc_prices[["open", "high", "low", "close", "volume", "adj close"]]
btc_prices.drop(columns="adj close", axis=1, inplace=True)
btc_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095


In [6]:
btc_only_prices = pd.concat([btc_prices], axis=1, keys=['BTC USD'])
btc_only_prices

BTC USD                                            \
                    open          high           low         close   
Date                                                                 
2020-01-01   7194.892090   7254.330566   7174.944336   7200.174316   
2020-01-02   7202.551270   7212.155273   6935.270020   6985.470215   
2020-01-03   6984.428711   7413.715332   6914.996094   7344.884277   
2020-01-04   7345.375488   7427.385742   7309.514160   7410.656738   
2020-01-05   7410.451660   7544.497070   7400.535645   7411.317383   
...                  ...           ...           ...           ...   
2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469   
2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938   
2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000   
2021-12-31  47169.371094  48472.527344  45819.953125  46306.445312   
2022-01-01  46311.746094  47827.312500  46288.484375  47686.812500   

                         
                 volume  
Date                     
2020-01-01  18565664997  
2020-01-02  20802083465  
2020-01-03  28111481032  
2020-01-04  18444271275  
2020-01-05  19725074095  
...                 ...  
2021-12-28  33430376883  
2021-12-29  30049226299  
2021-12-30  26686491018  
2021-12-31  36974172400  
2022-01-01  24582667004  

[732 rows x 5 columns]

In [7]:
iwp_prices = reader.get_data_yahoo(ticker2, start, end)
display(iwp_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWP,IWP,IWP,IWP,IWP,IWP
Date,,,,,,
2020-01-02,76.350685,77.074997,77.074997,76.400002,76.785004,891200.0
2020-01-03,76.048538,76.769997,76.934998,75.980003,76.224998,586600.0
2020-01-06,76.345711,77.070000,77.070000,76.110001,76.260002,777000.0
2020-01-07,76.330879,77.055000,77.245003,76.805000,76.980003,369800.0
2020-01-08,76.702347,77.430000,77.769997,76.974998,77.120003,412400.0


In [8]:
iwp_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iwp_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,76.350685,77.074997,77.074997,76.400002,76.785004,891200.0
2020-01-03,76.048538,76.769997,76.934998,75.980003,76.224998,586600.0
2020-01-06,76.345711,77.070000,77.070000,76.110001,76.260002,777000.0
2020-01-07,76.330879,77.055000,77.245003,76.805000,76.980003,369800.0
2020-01-08,76.702347,77.430000,77.769997,76.974998,77.120003,412400.0


In [9]:
iwp_prices = iwp_prices[["open", "high", "low", "close", "volume", "adj close"]]
iwp_prices.drop(columns="adj close", axis=1, inplace=True)
iwp_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,76.785004,77.074997,76.400002,77.074997,891200.0
2020-01-03,76.224998,76.934998,75.980003,76.769997,586600.0
2020-01-06,76.260002,77.070000,76.110001,77.070000,777000.0
2020-01-07,76.980003,77.245003,76.805000,77.055000,369800.0
2020-01-08,77.120003,77.769997,76.974998,77.430000,412400.0


In [10]:
iwp_only_prices = pd.concat([iwp_prices], axis=1, keys=['IWP'])
iwp_only_prices

IWP                                               
                  open        high         low       close     volume
Date                                                                 
2020-01-02   76.785004   77.074997   76.400002   77.074997   891200.0
2020-01-03   76.224998   76.934998   75.980003   76.769997   586600.0
2020-01-06   76.260002   77.070000   76.110001   77.070000   777000.0
2020-01-07   76.980003   77.245003   76.805000   77.055000   369800.0
2020-01-08   77.120003   77.769997   76.974998   77.430000   412400.0
...                ...         ...         ...         ...        ...
2021-12-27  115.139999  116.230003  115.010002  116.160004  1220000.0
2021-12-28  116.220001  116.650002  115.029999  115.279999   803700.0
2021-12-29  115.300003  115.690002  114.610001  115.540001   620900.0
2021-12-30  115.349998  116.699997  115.349998  115.669998   848800.0
2021-12-31  115.610001  116.050003  115.199997  115.220001   802200.0

[505 rows x 5 columns]

In [11]:
iws_prices = reader.get_data_yahoo(ticker3, start, end)
display(iws_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWS,IWS,IWS,IWS,IWS,IWS
Date,,,,,,
2020-01-02,90.591812,94.570000,95.209999,94.019997,95.160004,557000.0
2020-01-03,90.266129,94.230003,94.389999,93.650002,93.879997,443700.0
2020-01-06,90.294853,94.260002,94.269997,93.680000,93.830002,664600.0
2020-01-07,90.122429,94.080002,94.279999,93.839996,94.099998,358100.0
2020-01-08,90.160751,94.120003,94.480003,93.889999,94.169998,386400.0


In [12]:
iws_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iws_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,90.591812,94.570000,95.209999,94.019997,95.160004,557000.0
2020-01-03,90.266129,94.230003,94.389999,93.650002,93.879997,443700.0
2020-01-06,90.294853,94.260002,94.269997,93.680000,93.830002,664600.0
2020-01-07,90.122429,94.080002,94.279999,93.839996,94.099998,358100.0
2020-01-08,90.160751,94.120003,94.480003,93.889999,94.169998,386400.0


In [13]:
iws_prices = iws_prices[["open", "high", "low", "close", "volume", "adj close"]]
iws_prices.drop(columns="adj close", axis=1, inplace=True)
iws_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,95.160004,95.209999,94.019997,94.570000,557000.0
2020-01-03,93.879997,94.389999,93.650002,94.230003,443700.0
2020-01-06,93.830002,94.269997,93.680000,94.260002,664600.0
2020-01-07,94.099998,94.279999,93.839996,94.080002,358100.0
2020-01-08,94.169998,94.480003,93.889999,94.120003,386400.0


In [14]:
iws_only_prices = pd.concat([iws_prices], axis=1, keys=['IWS'])
iws_only_prices

IWS                                              
                  open        high         low       close    volume
Date                                                                
2020-01-02   95.160004   95.209999   94.019997   94.570000  557000.0
2020-01-03   93.879997   94.389999   93.650002   94.230003  443700.0
2020-01-06   93.830002   94.269997   93.680000   94.260002  664600.0
2020-01-07   94.099998   94.279999   93.839996   94.080002  358100.0
2020-01-08   94.169998   94.480003   93.889999   94.120003  386400.0
...                ...         ...         ...         ...       ...
2021-12-27  120.190002  121.720001  119.949997  121.720001  638000.0
2021-12-28  121.699997  122.419998  121.639999  121.989998  392500.0
2021-12-29  122.080002  122.500000  121.699997  122.389999  287700.0
2021-12-30  122.389999  123.040001  122.080002  122.190002  241200.0
2021-12-31  122.070000  122.900002  121.940002  122.379997  299400.0

[505 rows x 5 columns]

In [15]:
mgc_prices = reader.get_data_yahoo(ticker4, start, end)
display(mgc_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,MGC,MGC,MGC,MGC,MGC,MGC
Date,,,,,,
2020-01-02,109.172371,112.889999,112.889999,112.110001,112.379997,135600.0
2020-01-03,108.379372,112.070000,112.489998,111.529999,111.750000,69200.0
2020-01-06,108.833900,112.540001,112.550003,111.199997,111.379997,192200.0
2020-01-07,108.524445,112.220001,112.449997,112.099998,112.410004,75000.0
2020-01-08,109.143356,112.860001,113.300003,112.180000,112.180000,113600.0


In [16]:
mgc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
mgc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,109.172371,112.889999,112.889999,112.110001,112.379997,135600.0
2020-01-03,108.379372,112.070000,112.489998,111.529999,111.750000,69200.0
2020-01-06,108.833900,112.540001,112.550003,111.199997,111.379997,192200.0
2020-01-07,108.524445,112.220001,112.449997,112.099998,112.410004,75000.0
2020-01-08,109.143356,112.860001,113.300003,112.180000,112.180000,113600.0


In [17]:
mgc_prices = mgc_prices[["open", "high", "low", "close", "volume", "adj close"]]
mgc_prices.drop(columns="adj close", axis=1, inplace=True)
mgc_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,112.379997,112.889999,112.110001,112.889999,135600.0
2020-01-03,111.750000,112.489998,111.529999,112.070000,69200.0
2020-01-06,111.379997,112.550003,111.199997,112.540001,192200.0
2020-01-07,112.410004,112.449997,112.099998,112.220001,75000.0
2020-01-08,112.180000,113.300003,112.180000,112.860001,113600.0


In [18]:
mgc_only_prices = pd.concat([mgc_prices], axis=1, keys=['MGC'])
mgc_only_prices

MGC                                              
                  open        high         low       close    volume
Date                                                                
2020-01-02  112.379997  112.889999  112.110001  112.889999  135600.0
2020-01-03  111.750000  112.489998  111.529999  112.070000   69200.0
2020-01-06  111.379997  112.550003  111.199997  112.540001  192200.0
2020-01-07  112.410004  112.449997  112.099998  112.220001   75000.0
2020-01-08  112.180000  113.300003  112.180000  112.860001  113600.0
...                ...         ...         ...         ...       ...
2021-12-27  167.470001  169.259995  167.470001  169.229996   59400.0
2021-12-28  169.410004  169.679993  168.770004  169.000000   70500.0
2021-12-29  169.100006  169.470001  168.619995  169.160004   86800.0
2021-12-30  169.160004  169.710007  168.550003  168.710007   67100.0
2021-12-31  168.500000  168.889999  168.029999  168.080002   57000.0

[505 rows x 5 columns]

In [19]:
iwc_prices = reader.get_data_yahoo(ticker5, start, end)
display(iwc_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWC,IWC,IWC,IWC,IWC,IWC
Date,,,,,,
2020-01-02,96.959007,99.120003,100.370003,98.279999,100.370003,105100.0
2020-01-03,96.518814,98.669998,98.709999,98.099998,98.150002,21200.0
2020-01-06,96.616623,98.769997,99.050003,97.650002,97.970001,76100.0
2020-01-07,96.518814,98.669998,98.809998,98.250000,98.809998,16200.0
2020-01-08,96.636200,98.790001,99.239998,98.620003,98.660004,51800.0


In [20]:
iwc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iwc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2020-01-02,96.959007,99.120003,100.370003,98.279999,100.370003,105100.0
2020-01-03,96.518814,98.669998,98.709999,98.099998,98.150002,21200.0
2020-01-06,96.616623,98.769997,99.050003,97.650002,97.970001,76100.0
2020-01-07,96.518814,98.669998,98.809998,98.250000,98.809998,16200.0
2020-01-08,96.636200,98.790001,99.239998,98.620003,98.660004,51800.0


In [21]:
iwc_prices = iwc_prices[["open", "high", "low", "close", "volume", "adj close"]]
iwc_prices.drop(columns="adj close", axis=1, inplace=True)
iwc_prices.head()

,open,high,low,close,volume
Date,,,,,
2020-01-02,100.370003,100.370003,98.279999,99.120003,105100.0
2020-01-03,98.150002,98.709999,98.099998,98.669998,21200.0
2020-01-06,97.970001,99.050003,97.650002,98.769997,76100.0
2020-01-07,98.809998,98.809998,98.250000,98.669998,16200.0
2020-01-08,98.660004,99.239998,98.620003,98.790001,51800.0


In [22]:
iwc_only_prices = pd.concat([iwc_prices], axis=1, keys=['IWC'])
iwc_only_prices

IWC                                              
                  open        high         low       close    volume
Date                                                                
2020-01-02  100.370003  100.370003   98.279999   99.120003  105100.0
2020-01-03   98.150002   98.709999   98.099998   98.669998   21200.0
2020-01-06   97.970001   99.050003   97.650002   98.769997   76100.0
2020-01-07   98.809998   98.809998   98.250000   98.669998   16200.0
2020-01-08   98.660004   99.239998   98.620003   98.790001   51800.0
...                ...         ...         ...         ...       ...
2021-12-27  140.580002  141.529999  139.539993  141.429993  100000.0
2021-12-28  141.389999  142.000000  139.580002  139.860001   27500.0
2021-12-29  139.529999  140.000000  138.750000  139.250000   88900.0
2021-12-30  139.350006  141.330002  139.350006  139.729996   60600.0
2021-12-31  139.729996  140.500000  139.539993  139.729996  115000.0

[505 rows x 5 columns]

In [23]:
# Concatenate the ticker DataFrames
prices_btc_iwp = pd.concat([btc_prices, iwp_prices], axis=1, keys=["BTC-USD", "IWP"])
prices_btc_iws = pd.concat([btc_prices, iws_prices], axis=1, keys=["BTC-USD", "IWS"])
prices_btc_mgc = pd.concat([btc_prices, mgc_prices], axis=1, keys=["BTC-USD", "MGC"])
prices_btc_iwc = pd.concat([btc_prices, iwc_prices], axis=1, keys=["BTC-USD", "IWC"])

prices_btc_iwp = prices_btc_iwp.dropna()
prices_btc_iws = prices_btc_iws.dropna()
prices_btc_mgc = prices_btc_mgc.dropna()
prices_btc_iwc = prices_btc_iwc.dropna()

display(prices_btc_iwp.tail())
display(prices_btc_iws.tail())
display(prices_btc_mgc.tail())
prices_btc_iwc.tail()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2021-12-27  50802.609375  51956.328125  50499.468750  50640.417969   
2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469   
2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938   
2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000   
2021-12-31  47169.371094  48472.527344  45819.953125  46306.445312   

                                IWP                                      \
                 volume        open        high         low       close   
Date                                                                      
2021-12-27  24324345758  115.139999  116.230003  115.010002  116.160004   
2021-12-28  33430376883  116.220001  116.650002  115.029999  115.279999   
2021-12-29  30049226299  115.300003  115.690002  114.610001  115.540001   
2021-12-30  26686491018  115.349998  116.699997  115.349998  115.669998   
2021-12-31  36974172400  115.610001  116.050003  115.199997  115.220001   

                       
               volume  
Date                   
2021-12-27  1220000.0  
2021-12-28   803700.0  
2021-12-29   620900.0  
2021-12-30   848800.0  
2021-12-31   802200.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2021-12-27  50802.609375  51956.328125  50499.468750  50640.417969   
2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469   
2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938   
2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000   
2021-12-31  47169.371094  48472.527344  45819.953125  46306.445312   

                                IWS                                      \
                 volume        open        high         low       close   
Date                                                                      
2021-12-27  24324345758  120.190002  121.720001  119.949997  121.720001   
2021-12-28  33430376883  121.699997  122.419998  121.639999  121.989998   
2021-12-29  30049226299  122.080002  122.500000  121.699997  122.389999   
2021-12-30  26686491018  122.389999  123.040001  122.080002  122.190002   
2021-12-31  36974172400  122.070000  122.900002  121.940002  122.379997   

                      
              volume  
Date                  
2021-12-27  638000.0  
2021-12-28  392500.0  
2021-12-29  287700.0  
2021-12-30  241200.0  
2021-12-31  299400.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2021-12-27  50802.609375  51956.328125  50499.468750  50640.417969   
2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469   
2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938   
2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000   
2021-12-31  47169.371094  48472.527344  45819.953125  46306.445312   

                                MGC                                      \
                 volume        open        high         low       close   
Date                                                                      
2021-12-27  24324345758  167.470001  169.259995  167.470001  169.229996   
2021-12-28  33430376883  169.410004  169.679993  168.770004  169.000000   
2021-12-29  30049226299  169.100006  169.470001  168.619995  169.160004   
2021-12-30  26686491018  169.160004  169.710007  168.550003  168.710007   
2021-12-31  36974172400  168.500000  168.889999  168.029999  168.080002   

                     
             volume  
Date                 
2021-12-27  59400.0  
2021-12-28  70500.0  
2021-12-29  86800.0  
2021-12-30  67100.0  
2021-12-31  57000.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2021-12-27  50802.609375  51956.328125  50499.468750  50640.417969   
2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469   
2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938   
2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000   
2021-12-31  47169.371094  48472.527344  45819.953125  46306.445312   

                                IWC                                      \
                 volume        open        high         low       close   
Date                                                                      
2021-12-27  24324345758  140.580002  141.529999  139.539993  141.429993   
2021-12-28  33430376883  141.389999  142.000000  139.580002  139.860001   
2021-12-29  30049226299  139.529999  140.000000  138.750000  139.250000   
2021-12-30  26686491018  139.350006  141.330002  139.350006  139.729996   
2021-12-31  36974172400  139.729996  140.500000  139.539993  139.729996   

                      
              volume  
Date                  
2021-12-27  100000.0  
2021-12-28   27500.0  
2021-12-29   88900.0  
2021-12-30   60600.0  
2021-12-31  115000.0

# PART II: MONTE CARLOS 

In [24]:
#MC_btc = MCSimulation(
 #   portfolio_data = btc_only_prices,
  #  weights = [1],
   # num_simulation = 500,
    #num_trading_days = 126)

# Review the simulation input data
#MC_btc.portfolio_data.head()

In [25]:
MC_iwp = MCSimulation(
    portfolio_data = iwp_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_iwp.portfolio_data.head()

IWP                                                        
                 open       high        low      close    volume daily_return
Date                                                                         
2020-01-02  76.785004  77.074997  76.400002  77.074997  891200.0          NaN
2020-01-03  76.224998  76.934998  75.980003  76.769997  586600.0    -0.003957
2020-01-06  76.260002  77.070000  76.110001  77.070000  777000.0     0.003908
2020-01-07  76.980003  77.245003  76.805000  77.055000  369800.0    -0.000195
2020-01-08  77.120003  77.769997  76.974998  77.430000  412400.0     0.004867

In [26]:
MC_iws = MCSimulation(
    portfolio_data = iws_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_iws.portfolio_data.head()

IWS                                                        
                 open       high        low      close    volume daily_return
Date                                                                         
2020-01-02  95.160004  95.209999  94.019997  94.570000  557000.0          NaN
2020-01-03  93.879997  94.389999  93.650002  94.230003  443700.0    -0.003595
2020-01-06  93.830002  94.269997  93.680000  94.260002  664600.0     0.000318
2020-01-07  94.099998  94.279999  93.839996  94.080002  358100.0    -0.001910
2020-01-08  94.169998  94.480003  93.889999  94.120003  386400.0     0.000425

In [27]:
MC_mgc = MCSimulation(
    portfolio_data = mgc_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_mgc.portfolio_data.head()

MGC                                                \
                  open        high         low       close    volume   
Date                                                                   
2020-01-02  112.379997  112.889999  112.110001  112.889999  135600.0   
2020-01-03  111.750000  112.489998  111.529999  112.070000   69200.0   
2020-01-06  111.379997  112.550003  111.199997  112.540001  192200.0   
2020-01-07  112.410004  112.449997  112.099998  112.220001   75000.0   
2020-01-08  112.180000  113.300003  112.180000  112.860001  113600.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.007264  
2020-01-06     0.004194  
2020-01-07    -0.002843  
2020-01-08     0.005703

In [28]:
MC_iwc = MCSimulation(
    portfolio_data = iwc_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_iwc.portfolio_data.head()

IWC                                              \
                  open        high        low      close    volume   
Date                                                                 
2020-01-02  100.370003  100.370003  98.279999  99.120003  105100.0   
2020-01-03   98.150002   98.709999  98.099998  98.669998   21200.0   
2020-01-06   97.970001   99.050003  97.650002  98.769997   76100.0   
2020-01-07   98.809998   98.809998  98.250000  98.669998   16200.0   
2020-01-08   98.660004   99.239998  98.620003  98.790001   51800.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.004540  
2020-01-06     0.001013  
2020-01-07    -0.001012  
2020-01-08     0.001216

In [29]:
MC_btc_iwp = MCSimulation(
    portfolio_data = prices_btc_iwp,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_iwp.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                               IWP                                             \
           daily_return       open       high        low      close    volume   
Date                                                                            
2020-01-02          NaN  76.785004  77.074997  76.400002  77.074997  891200.0   
2020-01-03     0.051452  76.224998  76.934998  75.980003  76.769997  586600.0   
2020-01-06     0.057773  76.260002  77.070000  76.110001  77.070000  777000.0   
2020-01-07     0.050774  76.980003  77.245003  76.805000  77.055000  369800.0   
2020-01-08    -0.010269  77.120003  77.769997  76.974998  77.430000  412400.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.003957  
2020-01-06     0.003908  
2020-01-07    -0.000195  
2020-01-08     0.004867

In [30]:
MC_btc_iws = MCSimulation(
    portfolio_data = prices_btc_iws,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_iws.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                               IWS                                             \
           daily_return       open       high        low      close    volume   
Date                                                                            
2020-01-02          NaN  95.160004  95.209999  94.019997  94.570000  557000.0   
2020-01-03     0.051452  93.879997  94.389999  93.650002  94.230003  443700.0   
2020-01-06     0.057773  93.830002  94.269997  93.680000  94.260002  664600.0   
2020-01-07     0.050774  94.099998  94.279999  93.839996  94.080002  358100.0   
2020-01-08    -0.010269  94.169998  94.480003  93.889999  94.120003  386400.0   

                         
           daily_return  
Date                     
2020-01-02          NaN  
2020-01-03    -0.003595  
2020-01-06     0.000318  
2020-01-07    -0.001910  
2020-01-08     0.000425

In [31]:
MC_btc_mgc = MCSimulation(
    portfolio_data = prices_btc_mgc,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 252)

# Review the simulation input data
MC_btc_mgc.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                                MGC                                      \
           daily_return        open        high         low       close   
Date                                                                      
2020-01-02          NaN  112.379997  112.889999  112.110001  112.889999   
2020-01-03     0.051452  111.750000  112.489998  111.529999  112.070000   
2020-01-06     0.057773  111.379997  112.550003  111.199997  112.540001   
2020-01-07     0.050774  112.410004  112.449997  112.099998  112.220001   
2020-01-08    -0.010269  112.180000  113.300003  112.180000  112.860001   

                                   
              volume daily_return  
Date                               
2020-01-02  135600.0          NaN  
2020-01-03   69200.0    -0.007264  
2020-01-06  192200.0     0.004194  
2020-01-07   75000.0    -0.002843  
2020-01-08  113600.0     0.005703

In [32]:
MC_btc_iwc = MCSimulation(
    portfolio_data = prices_btc_iwc,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 126)

# Review the simulation input data
MC_btc_iwc.portfolio_data.head()

BTC-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2020-01-02  7202.551270  7212.155273  6935.270020  6985.470215  20802083465   
2020-01-03  6984.428711  7413.715332  6914.996094  7344.884277  28111481032   
2020-01-06  7410.452148  7781.867188  7409.292969  7769.219238  23276261598   
2020-01-07  7768.682129  8178.215820  7768.227539  8163.692383  28767291327   
2020-01-08  8161.935547  8396.738281  7956.774414  8079.862793  31672559265   

                                IWC                                    \
           daily_return        open        high        low      close   
Date                                                                    
2020-01-02          NaN  100.370003  100.370003  98.279999  99.120003   
2020-01-03     0.051452   98.150002   98.709999  98.099998  98.669998   
2020-01-06     0.057773   97.970001   99.050003  97.650002  98.769997   
2020-01-07     0.050774   98.809998   98.809998  98.250000  98.669998   
2020-01-08    -0.010269   98.660004   99.239998  98.620003  98.790001   

                                   
              volume daily_return  
Date                               
2020-01-02  105100.0          NaN  
2020-01-03   21200.0    -0.004540  
2020-01-06   76100.0     0.001013  
2020-01-07   16200.0    -0.001012  
2020-01-08   51800.0     0.001216

In [33]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
#MC_btc.calc_cumulative_return()

In [34]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iwp.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.021337,1.002443,1.000359,1.005846,0.999101,0.992757,0.984750,0.984496,1.003113,1.027179,...,0.993897,1.018962,1.005206,1.007647,0.998033,1.016864,0.994872,0.976329,1.023980,0.993444
2,1.003513,0.985115,1.001698,1.011082,0.993149,1.012900,0.949857,0.968631,1.000236,1.035100,...,0.967958,0.963799,0.974849,1.047096,0.983573,1.008711,1.036076,0.959111,1.059017,0.986470
3,0.968834,0.968971,1.017223,0.992322,1.025981,1.004616,0.951465,0.955732,0.989878,1.012846,...,0.927597,0.989894,0.979964,1.027913,0.993490,0.993043,1.029890,0.967856,1.051521,0.979887
4,0.952243,0.938408,1.002952,1.003310,1.029377,0.990146,0.959547,0.985877,0.971594,1.010555,...,0.921460,1.002919,0.974873,1.067800,0.978905,0.992202,1.020246,0.951189,1.048764,0.970129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.032424,1.189274,1.064869,2.144083,1.474281,1.029336,1.151205,2.174185,1.087238,1.999452,...,1.245143,1.521702,1.345386,2.145224,0.816071,1.677254,1.024525,1.326171,1.215854,2.368156
249,1.007357,1.198323,1.067368,2.130305,1.497803,1.029830,1.109449,2.182977,1.090350,2.056341,...,1.284465,1.553237,1.360273,2.198037,0.819069,1.722060,1.007736,1.383630,1.225026,2.442579
250,1.002168,1.215946,1.060159,2.159741,1.497903,1.012860,1.129369,2.181199,1.091617,2.060028,...,1.244107,1.514185,1.367858,2.235366,0.809887,1.743485,0.979688,1.417973,1.215466,2.472496
251,1.003946,1.198579,1.074052,2.198759,1.525958,1.011114,1.127162,2.204963,1.051823,2.040656,...,1.240906,1.484701,1.367717,2.270440,0.806190,1.691400,0.994888,1.406366,1.221064,2.509066


In [35]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iws.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.024248,1.019870,1.018354,1.032817,1.020890,1.019326,1.018595,1.020700,1.017079,1.003212,...,1.003742,0.999932,1.021258,0.962377,1.002688,0.982618,0.997868,0.999790,1.038974,0.999784
2,1.023268,1.034360,1.006801,1.022522,0.994043,0.984163,1.029392,0.990503,1.021986,0.988300,...,0.966520,1.017455,1.015921,0.970655,1.020004,0.954298,1.013704,0.997596,1.064059,0.992012
3,1.069117,1.045340,0.983040,1.000762,1.001652,0.978198,1.038917,0.983962,1.015836,0.981346,...,0.953732,1.019486,1.033006,0.977261,1.039074,0.964934,1.025435,0.995016,1.057058,1.013477
4,1.059159,1.037314,0.955451,1.037732,1.033957,0.958781,1.065259,0.982899,1.025013,0.994650,...,0.956495,1.005694,1.014748,0.968506,1.080365,0.982411,1.005171,1.028486,1.066074,1.015633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.925065,1.463857,1.251516,0.905925,1.023373,0.809366,1.524019,0.942625,1.533406,0.581010,...,1.192783,2.214208,1.072678,2.072242,0.962867,0.519429,1.027197,0.797412,0.891190,1.561807
249,1.979880,1.464965,1.277523,0.890992,0.997405,0.808806,1.505539,0.950782,1.538874,0.574658,...,1.203584,2.240002,1.047952,2.116004,0.983687,0.509057,1.028684,0.807448,0.906383,1.543778
250,1.974071,1.418628,1.280375,0.908728,1.023843,0.836768,1.513653,0.952249,1.530066,0.580991,...,1.214443,2.278023,1.019563,2.108846,0.986423,0.512636,1.036863,0.818953,0.911578,1.557437
251,2.023027,1.488346,1.296897,0.913263,1.003445,0.828729,1.552602,0.960027,1.499715,0.588199,...,1.200777,2.280320,1.042304,2.105446,0.963323,0.522554,1.023897,0.833876,0.909884,1.534141


In [36]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_mgc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.015261,1.008568,0.991174,1.009957,0.996207,0.987148,0.998535,1.036639,0.994056,0.985151,...,1.017195,1.005472,1.004267,0.999500,0.999246,1.013604,1.023893,1.013959,0.987115,1.003862
2,0.998492,1.006016,1.003262,1.027677,1.012678,0.990435,1.018071,1.054805,0.973321,0.968729,...,1.012089,0.996061,1.025629,0.974801,1.005232,1.024863,1.036229,1.031726,0.956152,0.991194
3,1.022500,0.994866,1.000337,1.039853,1.027677,1.004525,1.016529,1.044395,0.987517,0.972725,...,1.025456,1.010475,1.013802,1.002494,1.008122,0.987151,1.022480,1.028377,0.957737,1.025210
4,1.031382,1.001458,1.010190,1.020422,1.032878,1.009658,1.019846,1.058244,0.977944,0.956897,...,1.016440,1.001083,0.998248,1.038704,1.001694,0.972674,1.017676,1.015437,0.942800,1.051771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.959487,0.577519,1.263441,1.513144,1.638919,1.482524,1.618337,1.010341,1.985194,1.131815,...,1.067602,1.681638,1.446520,1.139735,1.281667,1.548117,1.337389,2.017622,1.100729,1.343516
249,0.954171,0.574562,1.281713,1.557255,1.667381,1.455226,1.625632,1.015034,1.942344,1.143579,...,1.057148,1.688649,1.437717,1.120034,1.206706,1.532919,1.379183,1.984510,1.074744,1.329367
250,0.951956,0.582758,1.270264,1.585712,1.667322,1.434767,1.616419,0.996070,1.931786,1.116089,...,1.053341,1.714407,1.393327,1.098477,1.213732,1.513070,1.368127,2.011141,1.079337,1.330674
251,0.944592,0.593719,1.278400,1.612990,1.702503,1.472102,1.622545,0.992262,1.920413,1.132404,...,1.041628,1.735223,1.394936,1.074703,1.177210,1.528666,1.385643,1.994653,1.083798,1.313523


In [37]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iwc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.961428,1.018513,0.995534,1.006135,0.990717,0.981458,0.996164,0.991906,0.984451,0.999218,...,0.975284,0.968676,0.982568,1.008497,1.027676,0.984030,1.000829,0.992566,1.033032,0.995746
2,0.948738,0.943077,1.006737,1.018615,0.991694,0.988909,1.018714,1.011876,0.974189,0.992778,...,0.928835,0.945238,1.031166,1.007407,1.050041,0.988504,1.014852,0.965714,1.051974,1.004428
3,0.935312,0.930306,1.030768,1.022259,0.964351,0.986730,1.018025,0.965726,0.956405,0.995363,...,0.938481,0.977398,1.059580,1.014406,1.086601,0.979450,1.017619,0.970485,1.061262,1.005277
4,0.930505,0.966674,1.026510,1.003769,0.939823,1.010702,0.994587,0.972109,0.933331,0.997038,...,0.919744,0.956342,1.056414,1.000335,1.083955,1.004402,1.014190,0.977974,1.068405,1.007766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0.829082,1.352765,1.693727,1.880213,1.187904,0.871560,1.236149,0.993298,1.173907,1.210546,...,1.363768,1.666299,1.521847,2.390549,0.991551,0.982666,2.048196,1.454795,1.208044,1.018797
249,0.817480,1.343152,1.731372,1.820896,1.217574,0.903869,1.287353,0.981654,1.194283,1.207319,...,1.403408,1.684463,1.501414,2.406462,0.978433,0.984454,1.989123,1.459823,1.198017,0.993973
250,0.812892,1.349872,1.716919,1.777924,1.224990,0.885104,1.328280,0.974318,1.229244,1.209669,...,1.369856,1.696276,1.550730,2.398350,0.993597,0.977994,2.003753,1.460887,1.238074,0.967053
251,0.825229,1.359211,1.689484,1.779779,1.213201,0.897461,1.344619,0.982067,1.243897,1.191058,...,1.387922,1.721157,1.556956,2.294553,1.000029,1.008916,1.966502,1.469184,1.229438,0.954350


In [38]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_btc_iwp.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.973941,0.987402,1.002417,1.007759,0.990585,0.977361,0.984242,0.996693,1.025828,1.014040,...,0.982000,0.980667,0.996256,0.999683,1.011985,0.990575,0.993251,1.020194,0.989712,0.992401
2,0.994081,0.986143,1.007251,1.006313,0.975429,0.964414,0.974914,1.000881,1.041119,1.034711,...,0.981413,0.979171,1.008208,0.996052,1.002725,0.994004,0.974063,1.025068,0.986776,0.975920
3,1.004907,0.981550,1.014619,1.008019,0.962555,0.968888,0.952907,0.999050,1.025199,1.048849,...,0.967302,0.964503,1.008033,0.987812,0.976673,0.991551,0.980158,1.033097,0.972139,0.987531
4,1.011503,0.958556,1.000290,0.991767,0.950544,0.964531,0.972244,1.033994,1.030613,1.053934,...,0.952117,1.005882,1.049589,1.017247,0.958680,0.982428,0.973250,1.037056,0.970430,0.976699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.337280,1.481543,1.850504,1.657086,2.281735,0.935908,1.493642,2.536824,1.165352,0.844126,...,0.999276,1.373111,1.621408,1.683274,1.447239,1.375315,1.134886,2.060326,1.300838,1.051096
249,1.307599,1.468683,1.889068,1.679709,2.229313,0.943709,1.490758,2.481045,1.169604,0.836333,...,1.010228,1.353211,1.608069,1.658421,1.444472,1.372374,1.127385,2.066007,1.255237,1.006326
250,1.289474,1.482493,1.876593,1.672968,2.182598,0.945022,1.509734,2.555593,1.155659,0.834473,...,1.017485,1.389631,1.617097,1.653306,1.437270,1.344117,1.129994,2.082291,1.251194,1.038284
251,1.265752,1.491987,1.916130,1.639219,2.145539,0.965059,1.519468,2.569079,1.155851,0.842068,...,1.021604,1.408184,1.644290,1.701088,1.407531,1.357376,1.125505,2.032707,1.248601,1.064500


In [39]:
MC_btc_iws.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.992183,1.004488,0.993647,0.989317,1.024504,0.997930,0.986372,0.976518,1.017564,1.006845,...,1.018895,1.003893,1.013528,1.006534,1.002696,0.989192,1.001729,1.017202,1.012338,0.998147
2,0.996447,1.017735,1.031568,0.986249,1.004244,0.990132,1.011285,0.973535,0.983913,1.018393,...,0.995996,1.008041,1.002134,0.995390,0.983671,0.976417,1.000772,1.012200,1.012503,0.989272
3,0.976881,1.004154,1.041710,0.971019,0.985567,0.977964,1.015909,0.997101,1.015656,1.024119,...,0.981894,0.979345,0.997724,0.995880,0.976935,0.971110,1.002281,0.999359,1.006683,1.001395
4,0.973155,0.970064,1.036890,0.977809,0.986518,0.971033,0.947857,1.003456,1.012863,1.026361,...,0.993504,1.013934,1.021747,0.997019,0.979259,0.982525,1.028061,1.001518,1.004369,1.001463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2.049480,1.319749,1.238804,0.812885,1.436077,1.901080,1.078287,1.002055,1.994061,2.189278,...,1.201498,2.102172,1.182944,1.138895,1.877147,1.430165,1.056805,0.971595,1.523255,1.716548
249,2.070415,1.300943,1.241328,0.803338,1.415850,1.864648,1.053457,1.006517,2.032272,2.174282,...,1.185726,2.052435,1.178435,1.166426,1.910267,1.424023,1.039211,0.999579,1.554772,1.761556
250,2.073512,1.343141,1.229256,0.809794,1.478931,1.885760,1.099978,1.008072,1.972414,2.138361,...,1.197498,2.084921,1.215554,1.158734,2.005084,1.412284,1.059133,0.996530,1.535548,1.762676
251,2.085229,1.335371,1.247616,0.815319,1.463583,1.909996,1.114571,1.040143,1.909799,2.129929,...,1.200788,2.100944,1.247859,1.179112,2.065325,1.433749,1.069185,1.022731,1.507573,1.750653


In [40]:
MC_btc_mgc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.017608,0.984105,0.999326,1.018733,0.992459,0.988661,1.005199,1.006883,1.010317,1.001323,...,1.017260,0.991619,0.986498,0.974158,0.999013,1.032380,1.003187,0.974576,1.010411,0.984945
2,1.026372,0.980454,1.013031,1.010682,1.006077,0.990194,1.011201,1.017416,1.017363,1.039900,...,0.994744,0.995433,0.985960,0.973658,1.008631,1.006650,0.986176,0.981527,1.042339,0.997803
3,1.011477,0.986095,1.000345,1.008696,0.990194,0.973825,1.007420,1.034629,1.025354,1.040965,...,1.005345,0.974690,0.999059,0.980462,1.014644,0.989235,0.963509,0.980550,1.042257,1.007532
4,1.011046,0.998557,1.002754,1.004547,0.973981,0.977124,0.994110,1.047543,1.049818,1.025833,...,1.040399,0.992990,0.988390,0.965829,1.014457,0.982790,0.948866,0.983762,1.018306,1.008642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1.646697,1.378509,1.078436,1.159281,1.053787,0.946995,1.240741,1.165014,1.250226,1.382318,...,1.288458,1.371784,0.995648,1.763489,0.920026,2.146852,1.096485,1.026923,1.285806,1.114728
249,1.650861,1.372203,1.076130,1.178814,1.041833,0.924312,1.250449,1.184617,1.251509,1.396979,...,1.280037,1.368585,0.983403,1.783540,0.956660,2.158842,1.136926,1.015220,1.284059,1.103584
250,1.627499,1.375022,1.089946,1.198666,1.047708,0.936491,1.264730,1.154513,1.264671,1.384829,...,1.272618,1.383424,0.958372,1.740975,0.943755,2.173971,1.138978,1.027154,1.287038,1.147314
251,1.654857,1.389140,1.089709,1.211231,1.029688,0.928862,1.296764,1.132432,1.257415,1.339121,...,1.302964,1.397088,0.967922,1.735285,0.977341,2.151495,1.107066,1.035913,1.283013,1.150717


In [41]:
MC_btc_iwc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.996036,0.984587,1.002547,1.038176,1.005228,1.022814,0.984936,0.977930,1.031320,0.962047,...,0.996621,0.979881,0.975269,1.012922,1.020773,1.009308,0.984532,0.973516,0.995588,1.022266
2,0.999134,0.985550,0.975559,1.070880,0.997399,1.021180,0.950935,0.986154,1.035257,0.991069,...,1.003007,0.966094,0.989996,0.995925,1.053716,1.005916,0.982206,0.973462,1.018092,1.017596
3,1.013462,1.013750,0.964468,1.062171,0.980901,1.032293,0.959600,0.978202,1.033655,0.938584,...,1.006913,0.961139,1.021641,0.955761,1.047388,0.969845,1.000707,0.960424,1.028496,1.017284
4,1.039850,1.036465,0.969546,1.084523,0.974720,1.057107,1.006467,1.000824,1.033843,0.932942,...,1.028099,0.931292,1.008984,0.985465,1.041841,1.014691,1.003102,0.937378,1.020091,1.021249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,1.127062,1.138962,1.294338,1.601912,1.700630,1.028694,1.038725,1.338235,1.265410,1.031984,...,1.640758,0.891727,0.907155,1.106197,1.944567,1.673136,1.140423,0.945166,1.424190,1.521616
123,1.112653,1.120081,1.283689,1.619226,1.702025,1.041147,1.037971,1.294144,1.269655,1.008672,...,1.641280,0.904490,0.920493,1.086642,1.983155,1.677436,1.170042,0.929162,1.421642,1.569599
124,1.136450,1.117835,1.279305,1.656816,1.712548,1.051728,1.051576,1.297041,1.302224,0.998900,...,1.651237,0.914674,0.939602,1.057730,2.009030,1.669944,1.189934,0.956128,1.402561,1.578851
125,1.144647,1.100589,1.276321,1.667109,1.709478,1.061096,1.042897,1.307363,1.328344,0.976563,...,1.689142,0.883139,0.951605,1.093119,2.044711,1.660633,1.196524,0.954805,1.419433,1.602951


In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
#btc_line_plot = MC_btc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iwp_line_plot = MC_iwp.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iws_line_plot = MC_iws.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
mgc_line_plot = MC_mgc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iwc_line_plot = MC_iwc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iwp_line_plot = MC_btc_iwp.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iws_line_plot = MC_btc_iws.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_mgc_line_plot = MC_btc_mgc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iwc_line_plot = MC_btc_iwc.plot_simulation()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
#btc_distribution_plot = MC_btc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iwp_distribution_plot = MC_iwp.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iws_distribution_plot = MC_iws.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
mgc_distribution_plot = MC_mgc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iwc_distribution_plot = MC_iwc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iwp_distribution_plot = MC_btc_iwp.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iws_distribution_plot = MC_btc_iws.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_mgc_distribution_plot = MC_btc_mgc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iwc_distribution_plot = MC_btc_iwc.plot_distribution()

In [ ]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
#btc_table = MC_btc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
#print(btc_table)

In [42]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iwp_table = MC_iwp.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iwp_table)

count           1000.000000
mean               1.281110
std                0.383360
min                0.400792
25%                1.008303
50%                1.231908
75%                1.501858
max                3.082284
95% CI Lower       0.669653
95% CI Upper       2.185679
Name: 252, dtype: float64


In [43]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iws_table = MC_iws.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iws_table)

count           1000.000000
mean               1.187811
std                0.357664
min                0.459822
25%                0.925799
50%                1.143598
75%                1.394182
max                3.422702
95% CI Lower       0.629430
95% CI Upper       1.970758
Name: 252, dtype: float64


In [44]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
mgc_table = MC_mgc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(mgc_table)

count           1000.000000
mean               1.248291
std                0.322217
min                0.426861
25%                1.019543
50%                1.223608
75%                1.445892
max                2.685605
95% CI Lower       0.712238
95% CI Upper       1.966096
Name: 252, dtype: float64


In [45]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iwc_table = MC_iwc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iwc_table)

count           1000.000000
mean               1.261354
std                0.439458
min                0.364828
25%                0.952416
50%                1.191800
75%                1.485694
max                4.214516
95% CI Lower       0.619133
95% CI Upper       2.323437
Name: 252, dtype: float64


In [46]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iwp_table = MC_btc_iwp.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iwp_table)

count           1000.000000
mean               1.397266
std                0.392299
min                0.516320
25%                1.127035
50%                1.365115
75%                1.611278
max                4.009315
95% CI Lower       0.768322
95% CI Upper       2.265805
Name: 252, dtype: float64


In [47]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iws_table = MC_btc_iws.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iws_table)

count           1000.000000
mean               1.327247
std                0.391026
min                0.498239
25%                1.056649
50%                1.263096
75%                1.556080
max                3.722185
95% CI Lower       0.721182
95% CI Upper       2.208078
Name: 252, dtype: float64


In [48]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_mgc_table = MC_btc_mgc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_mgc_table)

count           1000.000000
mean               1.392834
std                0.337097
min                0.583311
25%                1.157104
50%                1.350251
75%                1.575985
max                3.001885
95% CI Lower       0.847200
95% CI Upper       2.157704
Name: 252, dtype: float64


In [49]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iwc_table = MC_btc_iwc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iwc_table)

count           1000.000000
mean               1.176010
std                0.272812
min                0.561643
25%                0.972509
50%                1.134247
75%                1.337299
max                2.400670
95% CI Lower       0.743515
95% CI Upper       1.788449
Name: 126, dtype: float64


In [50]:
# Print the current balance of the stock and bond portion of the members portfolio
investment = 100000

In [51]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
#ci_lower_cumulative_return = round(btc_table[8] * investment, 2)
#ci_upper_cumulative_return = round(btc_table[9] * investment, 2)

# Print the result of your calculations
#print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
 #     f" with a 100% BTC over the next six months will end within the range of"
#      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

#mean = round(btc_table[1] * investment, 2)
#print(mean)

In [52]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iwp_table[8] * investment, 2)
ci_upper_cumulative_return = round(iwp_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWP over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iwp_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWP over the next six months will end within the range of $66965.29 and $218567.88.
128111.0


In [53]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iws_table[8] * investment, 2)
ci_upper_cumulative_return = round(iws_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWS over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iws_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWS over the next six months will end within the range of $62943.01 and $197075.82.
118781.13


In [54]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(mgc_table[8] * investment, 2)
ci_upper_cumulative_return = round(mgc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% MGC over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(mgc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% MGC over the next six months will end within the range of $71223.84 and $196609.58.
124829.12


In [55]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iwc_table[8] * investment, 2)
ci_upper_cumulative_return = round(iwc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWC over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iwc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWC over the next six months will end within the range of $61913.27 and $232343.72.
126135.36


In [56]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iwp_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iwp_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWP respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iwp_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWP respectively over the next six months will end within the range of $76832.19 and $226580.5.
139726.57


In [57]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iws_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iws_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWS respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iws_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWS respectively over the next six months will end within the range of $72118.22 and $220807.79.
132724.74


In [58]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_mgc_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_mgc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and MGC respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_mgc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and MGC respectively over the next six months will end within the range of $84720.0 and $215770.36.
139283.42


In [59]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iwc_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iwc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWC respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iwc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWC respectively over the next six months will end within the range of $74351.5 and $178844.92.
117600.98
